# Multi-agent SIPP

In [1]:
import networkx as nx
from matplotlib import pyplot as plt
import numpy as np

import sys
sys.path.append('../')

sys.path.append('../../')

from panav.util import interpolate_positions

from panav.SIPP import compute_safe_intervals, plan_to_transitions,SIPP

from matplotlib.animation import FuncAnimation

from IPython.display import HTML, display


from panav.viz import animate_MAPF_R

%load_ext autoreload
%autoreload 2

In [2]:
node_locs = np.array([[0,1],
                      [0,-1],
                      [1,0],
                      [2,0],
                      [3,0],
                      [4,0],
                      [5,1],
                      [5,-1]])
N = len(node_locs)

G = nx.DiGraph()
G.add_nodes_from(range(N))
edges = [(0,2),
          (1,2),
          (2,3),
          (3,4),
          (4,5),
          (5,6),
          (5,7)]

G.add_edges_from(edges)
G.add_edges_from(e[::-1] for e in edges)

# We manually add the bi-directional edges here, because
# using nx.to_directed(a undirected graph) causes issue when accessing edge values.

In [3]:
starts = [0,1]
goals = [6,7]

v_max = 1.0
bloating_r = 0.2

nx.set_edge_attributes(G,
                       {e:np.linalg.norm(node_locs[e[0]]-node_locs[e[1]])/v_max\
                          for e in G.edges},
                       'weight')

In [4]:
def simple_path_to_plan(G_path):
    start_node = G_path[0]
    goal_node = G_path[-1]

    G_plan = [(start_node,0)]

    agent_loc = node_locs[start_node]
    t = 0
    wait_pos = 3
    wait_t = 3.5
    for i in range(len(G_path)-1):
        t+=G.edges[G_path[i],G_path[i+1]]['weight']
        G_plan.append((G_path[i+1],t))
        if i==wait_pos:
            t+=wait_t
            G_plan.append((G_path[i+1],t))


    return G_plan

In [5]:
G_path = [0,2,3,4,5,6]

plan = simple_path_to_plan(G_path)

In [6]:
agent_path = [1,2,3,4,5,7]

agent_plan = simple_path_to_plan(agent_path)


In [12]:
safe_plan = SIPP(G,node_locs,starts[1],goals[1], plan_to_transitions(plan), v_max, bloating_r)

In [18]:
from panav.MAPFR_conflict import MAPFR_conflict

In [19]:
MAPFR_conflict([safe_plan,plan],node_locs,bloating_r)

In [20]:
MAPFR_conflict([agent_plan,plan],node_locs,bloating_r)

{'agents': (0, 1),
 'conflict': [{'agent': 0,
   'transition': (1, 2),
   'duration': (0, 1.4142135623730951)},
  {'agent': 1, 'transition': (0, 2), 'duration': (0, 1.4142135623730951)}]}